In [2]:
import os
os.environ['SERPAPI_API_KEY']=""
os.environ['OPENAI_API_KEY']=""
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="LangGraph"
os.environ["LANGCHAIN_API_KEY"]=""


In [3]:
from langchain_community.utilities import SerpAPIWrapper

In [6]:
%pip install google-search-results

  Using cached google_search_results-2.4.2.tar.gz (18 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=b39c1d81d3995d2ff914bf3a28f31b56abdff26a07cf4432dd99dcaaa051e2b7
  Stored in directory: /Users/yanglinshuo/Library/Caches/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results
Note: you may need to restart the kernel to use updated packages.


In [4]:
search=SerpAPIWrapper()
search.run("Obama's first name?")

'Barack Hussein Obama II'

In [7]:
%pip install langgraph

  Using cached langgraph-0.2.4-py3-none-any.whl.metadata (13 kB)
  Using cached langgraph_checkpoint-1.0.3-py3-none-any.whl.metadata (4.6 kB)
Using cached langgraph-0.2.4-py3-none-any.whl (81 kB)
Using cached langgraph_checkpoint-1.0.3-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import functools, operator, requests, json
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict

In [10]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [11]:
from langchain_core.messages import (AIMessage, BaseMessage, ChatMessage,FunctionMessage, HumanMessage, SystemMessage)

@tool("web_search")
def web_search(query: str) -> str:
    """Search with Google SERP API by a query"""
    search = SerpAPIWrapper()
    return search.run(query)

@tool("twitter_writer")
def write_tweet(content: str) -> str:
    """Based a piece of content, write a tweet."""
    chat = ChatOpenAI(model="gpt-4o-mini")
    messages = [
        SystemMessage(
            content="You are a Twitter account operator. You a responsible for writting a tweet only based on the content given. You should follow the Twitter policy and make sure each tweet has no more than 140 characters."
        ),
        HumanMessage(
            content=content
        ),

    ]
    response = chat(messages)
    return response.content

In [12]:
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

In [13]:
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    excutor = AgentExecutor(agent=agent, tools=tools)
    return excutor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [15]:
members = ["Search_Engine", "Twitter_Writer"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the following workers: {members}."
    " Given the following user requests, respond with the worker to act next. Each worker will perform a task and respond with their results and status."
    " Whtn finished, respond with FINISH."
)

options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters":{
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next":{
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "system_prompt"),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}",),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [16]:
search_engine_agent = create_agent(llm, [web_search], "You are a web search engine.")
search_engine_node = functools.partial(agent_node, agent=search_engine_agent, name="Search_Engine")

twitter_operator_agent = create_agent(llm, [write_tweet], "You are responsible for writting a tweet based on the content given.")
twitter_operator_node = functools. partial(agent_node, agent=twitter_operator_agent, name="Twitter_Writer")

workflow = StateGraph(AgentState)
workflow.add_node("Search_Engine", search_engine_node)
workflow.add_node("Twitter_Writer", twitter_operator_node)
workflow.add_node("supervisor", supervisor_chain)

In [17]:
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"],conditional_map)
workflow.set_entry_point("supervisor")
graph = workflow.compile()

In [18]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Write a tweet about LangChain news.")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Twitter_Writer'}}
----


/Users/yanglinshuo/anaconda3/envs/streamlit/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'Twitter_Writer': {'messages': [HumanMessage(content='🚀 Exciting news! LangChain just dropped a major update enhancing API integration & boosting performance. A must for developers! #LangChain #AI #DevNews', name='Twitter_Writer')]}}
----
{'supervisor': {'next': 'FINISH'}}
----
